---

### Amaç:

* Normal bir CNN bloğunda nn.Conv2d yerine DepthwiseSeparableConv koymayı görmek

* Parametre sayısı farkını görmek

* Forward çalıştığını görmek

----


# 1) Önce: Depthwise + Pointwise Bloğunu Tanımlayalım

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
import torch
import torch.nn as nn

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, c_in, c_out, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.depthwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_in,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=c_in,
            bias=False
        )
        self.pointwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_out,
            kernel_size=1,
            bias=False
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

* Bu bloğu, normal Conv2d(cin, cout, 3x3) yerine kullanacağız.

## 2) Karşılaştırma İçin İki Küçük Model

* TinyCNNStandard → Tamamen normal Conv2d kullanan model

* TinyCNNDepthwise → Aynı yapının yerine DepthwiseSeparableConv koyulmuş hali

Giriş olarak CIFAR-10 tarzı 3×32×32 düşünebiliriz

In [5]:
class TinyCNNStandard(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # 32x32 -> pool -> 16x16 -> pool -> 8x8
        self.fc = nn.Linear(64 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))          # (N, 32, 16, 16)
        x = self.pool(F.relu(self.conv2(x)))          # (N, 64, 8, 8)
        x = x.view(x.size(0), -1)                     # (N, 64*8*8)
        x = self.fc(x)
        return x


class TinyCNNDepthwise(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = DepthwiseSeparableConv(3, 32, kernel_size=3, padding=1)
        self.conv2 = DepthwiseSeparableConv(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(64 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))          # (N, 32, 16, 16)
        x = self.pool(F.relu(self.conv2(x)))          # (N, 64, 8, 8)
        x = x.view(x.size(0), -1)                     # (N, 64*8*8)
        x = self.fc(x)
        return x

Burada entegrasyon net:

* Daha önce nn.Conv2d(cin, cout, 3, padding=1) yazdığımız yere

- Artık DepthwiseSeparableConv(cin, cout, 3, padding=1) yazıyoruz.
* Başka hiçbir şey değişmiyor.

## 3) Küçük Test: Parametre Sayısı ve Forward

Şimdi her iki modeli instantiate edip:

Toplam parametre sayısına bakalım

Rastgele bir girdiyle forward çalıştıralım

In [6]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

std_model = TinyCNNStandard(num_classes=10)
dw_model  = TinyCNNDepthwise(num_classes=10)

print("Standard param:", count_params(std_model))
print("Depthwise param:", count_params(dw_model))

x = torch.randn(4, 3, 32, 32)
y_std = std_model(x)
y_dw  = dw_model(x)

print("y_std shape:", y_std.shape)
print("y_dw  shape:", y_dw.shape)


Standard param: 60362
Depthwise param: 43429
y_std shape: torch.Size([4, 10])
y_dw  shape: torch.Size([4, 10])
